In [25]:
import boto3
import pandas as pd
from io import BytesIO

In [3]:
# Config cliente de S3
s3 = boto3.client('s3')

In [4]:
# Bbucket y prefijo de carpetas
bucket_name = 'viamericas-datalake-dev-us-east-1-283731589572-analytics'
prefix = 'prueba_sk_forecast/'

In [5]:
# Obtengo lista de carpetas en el bucket
def list_folders(bucket_name, prefix):
    response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix, Delimiter='/')
    folders = [prefix['Prefix'].split('/')[-2] for prefix in response.get('CommonPrefixes', [])]
    return folders

In [6]:
# Llamo funcion para listar carpetas
folders = list_folders(bucket_name, prefix)

In [7]:
len(folders)

124

In [23]:
# Leo los xlsx y consolido

def read_files(bucket_name, prefix):
    dfs = []
    
    # Iterar sobre cada carpeta
    for folder_name in folders:
        # Obtener la lista de objetos en la carpeta
        objects = s3.list_objects_v2(Bucket=bucket_name, Prefix=f'{prefix}{folder_name}/')['Contents']
        
        # Buscar archivos xlsx en la carpeta
        excel_objects = [obj for obj in objects if obj['Key'].endswith('.xlsx')]
        
        if excel_objects:
            # Leer el primer archivo xlsx encontrado
            obj = s3.get_object(Bucket=bucket_name, Key=excel_objects[0]['Key'])
            excel_data = obj['Body'].read() # No se puede leer con pandas directo de S3
            
            df = pd.read_excel(BytesIO(excel_data))
            # Agrego columna con el nombre de la carpeta
            df['folder_name'] = folder_name
            # Agregar el df a la lista
            dfs.append(df)
        else:
            print(f"No se encontraron archivos xlsx en la carpeta {folder_name}.")
    
    if dfs:
        # Concateno todo
        consolidated_df = pd.concat(dfs, ignore_index=True)
        return consolidated_df
    else:
        print("No se encontraron archivos xlsx en ninguna de las carpetas especificadas.")
        return None


In [26]:
# Llamo funcion para leer xlsx y consolidar en un unico DF
final_df = read_files(bucket_name, prefix)

No se encontraron archivos xlsx en la carpeta ABANK (TN)_EL SALVADOR.
No se encontraron archivos xlsx en la carpeta AFRO INTERNACIONAL_SIERRA LEONE.
No se encontraron archivos xlsx en la carpeta BANCO PICHINCHA (TN)_ECUADOR.
No se encontraron archivos xlsx en la carpeta GRUPO EXITO (TN)_COLOMBIA.
No se encontraron archivos xlsx en la carpeta MASTERCARD ATM NETWORK_UNITED STATES (ATM & DEPOSITS).
No se encontraron archivos xlsx en la carpeta TELEDOLAR-NIC (TN)_NICARAGUA.
No se encontraron archivos xlsx en la carpeta UNITRANSFER_HAITI.


In [28]:
#final_df.to_excel('Consolidado_SkForecast.xlsx')

In [29]:
def export_to_s3(df, bucket_name, prefix):
    # Convertir el DataFrame a un archivo Excel en memoria
    excel_buffer = BytesIO()
    df.to_excel(excel_buffer, index=False)
    
    # Crear un cliente de Amazon S3
    s3 = boto3.client('s3')
    
    # Subir el archivo Excel al bucket de S3
    file_name = "Consolidado_SkForecast.xlsx"
    file_key = f"{prefix}{file_name}"
    s3.put_object(Bucket=bucket_name, Key=file_key, Body=excel_buffer.getvalue())
    
    print(f"El archivo {file_name} ha sido exportado a {bucket_name}/{file_key}")

In [31]:
export_to_s3(final_df, bucket_name, prefix)

El archivo Consolidado_SkForecast.xlsx ha sido exportado a viamericas-datalake-dev-us-east-1-283731589572-analytics/prueba_sk_forecast/Consolidado_SkForecast.xlsx
